In [ ]:
import cv2
import time
import numpy as np
import os

MODE = "COCO"

protoFile = "pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "pose/coco/pose_iter_440000.caffemodel"
nPoints = 18
POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

directory = '../../Images/AugementedExample/' #hardcode change input foler
outDir = '../../Images/AugementedExampleCheck/' #hardcode change output foler
startTime = time.time()

image_num = 0
for filename in os.scandir(directory):
    if (filename.path.endswith(".jpg") or filename.path.endswith(".png")) and filename.is_file():
        print('Dtecting Kps on image: ' + str(image_num))
        frame = cv2.imread(filename.path)
        frameCopy = np.copy(frame)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        threshold = 0.1
#         threshold = 0.5 #moet getest worden

        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

        t = time.time()
        # input image dimensions for the network
        inWidth = 640
        inHeight = 480
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)

        output = net.forward()
        print("time taken by network : {:.3f}".format(time.time() - t))

        H = output.shape[2]
        W = output.shape[3]

        # Empty list to store the detected keypoints
        points = []

        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

                # Add the point to the list if the probability is greater than the threshold
                points.append((int(x), int(y)))
            else :
                points.append(None)

        # Draw Skeleton
        for pair in POSE_PAIRS:
            partA = pair[0]
            partB = pair[1]

            if points[partA] and points[partB]:
                cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
                cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                
        if not os.path.exists(outDir): #hardcode change output foler
            os.makedirs(outDir) 
            print('folder created')
            
        cv2.imwrite(outDir + os.path.basename(filename.path),frame)
        image_num +=1
print("Total time taken : {:.3f}".format(time.time() - startTime))

print("Done with Exporting")